#### Note(Credit to ):
    The code is learned from (Thank you for those who post the reference in the discussion board)
    https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/
    https://towardsdatascience.com/multi-layer-neural-networks-with-sigmoid-function-deep-learning-for-rookies-2-bf464f09eb7f
    https://machinelearningmastery.com/neural-networks-crash-course/

In [1]:
from random import random
from math import exp
from random import randrange

In [2]:
#read files
from csv import reader
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

#convert string to float
def str_column_to_float(dataset):
    data = []
    for row in dataset:
        row = list(map(float, row))
        data.append(row)
    return data

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

### Initialize Network

In [3]:
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network  

### Forward Propagation
Calculating an output from a neural network by propagting an input signal through each layer until the output layer outputs its value.

We can break forward propagation down into three parts:

1. Neuron Activation.
2. Neuron Transfer.
3. Forward Propagation.

In [4]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
#         print(weights[i])
#         print (inputs[i])
#         print(type(weights[i]), type( inputs[i]))
        activation += weights[i] * inputs[i]
        
    return activation

# Transfer neuron activation (activation function)
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
#     print('row:', row)
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
#     print(inputs)
    return inputs

### Back Propagatiton

Error is calculated between the expected outputs and the outputs forward propagated from the network. These errors are then propagated backward through the network from the output layer to the hidden layer.

In [5]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
#         print('network', network)
        layer = network[i]
#         print ('layer', layer)
        errors = list()
        if i != len(network)-1:
#             print(len(network))
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
#                     print('network', network)
#                     print("neuron",neuron)
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
                
        for j in range(len(layer)):
#             print(layer)
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

### 4. Train Network
The network is trained using stochastic gradient descent.


This part is broken down into two sections:

1. Update Weights.
2. Train Network.

In [6]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']
 
# 
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[int(row[-1])] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [7]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return(predictions)

# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

In [8]:
df = load_csv('RedWhiteWine.csv')
df.pop(0)
data = str_column_to_float(df)

In [9]:
data


[[7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978, 3.51, 0.56, 9.4, 5.0, 1.0],
 [7.8, 0.88, 0.0, 2.6, 0.098, 25.0, 67.0, 0.9968, 3.2, 0.68, 9.8, 5.0, 1.0],
 [7.8, 0.76, 0.04, 2.3, 0.092, 15.0, 54.0, 0.997, 3.26, 0.65, 9.8, 5.0, 1.0],
 [11.2, 0.28, 0.56, 1.9, 0.075, 17.0, 60.0, 0.998, 3.16, 0.58, 9.8, 6.0, 1.0],
 [7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978, 3.51, 0.56, 9.4, 5.0, 1.0],
 [7.4, 0.66, 0.0, 1.8, 0.075, 13.0, 40.0, 0.9978, 3.51, 0.56, 9.4, 5.0, 1.0],
 [7.9, 0.6, 0.06, 1.6, 0.069, 15.0, 59.0, 0.9964, 3.3, 0.46, 9.4, 5.0, 1.0],
 [7.3, 0.65, 0.0, 1.2, 0.065, 15.0, 21.0, 0.9946, 3.39, 0.47, 10.0, 7.0, 1.0],
 [7.8, 0.58, 0.02, 2.0, 0.073, 9.0, 18.0, 0.9968, 3.36, 0.57, 9.5, 7.0, 1.0],
 [7.5, 0.5, 0.36, 6.1, 0.071, 17.0, 102.0, 0.9978, 3.35, 0.8, 10.5, 5.0, 1.0],
 [6.7, 0.58, 0.08, 1.8, 0.097, 15.0, 65.0, 0.9959, 3.28, 0.54, 9.2, 5.0, 1.0],
 [7.5, 0.5, 0.36, 6.1, 0.071, 17.0, 102.0, 0.9978, 3.35, 0.8, 10.5, 5.0, 1.0],
 [5.6, 0.615, 0.0, 1.6, 0.089, 16.0, 59.0, 0.9943, 3.58, 0.52, 

In [9]:
# evaluate algorithm
n_folds = 5
l_rate = 0.001
n_epoch = 5
n_hidden = 5
scores = evaluate_algorithm(data, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/(len(scores))))

>epoch=0, lrate=0.001, error=4112.075
>epoch=1, lrate=0.001, error=2300.910
>epoch=2, lrate=0.001, error=1966.410
>epoch=3, lrate=0.001, error=1927.743
>epoch=4, lrate=0.001, error=1918.331
>epoch=0, lrate=0.001, error=3713.870
>epoch=1, lrate=0.001, error=2204.262
>epoch=2, lrate=0.001, error=2041.410
>epoch=3, lrate=0.001, error=1981.970
>epoch=4, lrate=0.001, error=1948.524
>epoch=0, lrate=0.001, error=3981.796
>epoch=1, lrate=0.001, error=2234.811
>epoch=2, lrate=0.001, error=1960.202
>epoch=3, lrate=0.001, error=1949.828
>epoch=4, lrate=0.001, error=1948.972
>epoch=0, lrate=0.001, error=3129.222
>epoch=1, lrate=0.001, error=1981.194
>epoch=2, lrate=0.001, error=1915.687
>epoch=3, lrate=0.001, error=1909.231
>epoch=4, lrate=0.001, error=1908.356
>epoch=0, lrate=0.001, error=4937.231
>epoch=1, lrate=0.001, error=4826.671
>epoch=2, lrate=0.001, error=4558.803
>epoch=3, lrate=0.001, error=3247.333
>epoch=4, lrate=0.001, error=2006.070
Scores: [74.51886066204773, 75.98152424942263, 76.

In [11]:
# evaluate algorithm with increased learning rate
n_folds = 5
l_rate = 1
n_epoch = 5
n_hidden = 5
scores = evaluate_algorithm(data, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/(len(scores))))

>epoch=0, lrate=1.000, error=2120.129
>epoch=1, lrate=1.000, error=2107.439
>epoch=2, lrate=1.000, error=2107.439
>epoch=3, lrate=1.000, error=2107.439
>epoch=4, lrate=1.000, error=2107.439
>epoch=0, lrate=1.000, error=2156.385
>epoch=1, lrate=1.000, error=2151.874
>epoch=2, lrate=1.000, error=2151.874
>epoch=3, lrate=1.000, error=2151.874
>epoch=4, lrate=1.000, error=2151.874
>epoch=0, lrate=1.000, error=2139.623
>epoch=1, lrate=1.000, error=2134.177
>epoch=2, lrate=1.000, error=2134.177
>epoch=3, lrate=1.000, error=2134.177
>epoch=4, lrate=1.000, error=2134.177
>epoch=0, lrate=1.000, error=2101.875
>epoch=1, lrate=1.000, error=2098.051
>epoch=2, lrate=1.000, error=2098.051
>epoch=3, lrate=1.000, error=2098.051
>epoch=4, lrate=1.000, error=2098.051
>epoch=0, lrate=1.000, error=2105.726
>epoch=1, lrate=1.000, error=2091.982
>epoch=2, lrate=1.000, error=2091.982
>epoch=3, lrate=1.000, error=2091.982
>epoch=4, lrate=1.000, error=2091.982
Scores: [74.82678983833718, 77.36720554272517, 76.

In [12]:
# evaluate algorithm with increased epoch
n_folds = 5
l_rate = 0.001
n_epoch = 100
n_hidden = 5
scores = evaluate_algorithm(data, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/(len(scores))))

>epoch=0, lrate=0.001, error=3847.304
>epoch=1, lrate=0.001, error=2168.020
>epoch=2, lrate=0.001, error=1962.910
>epoch=3, lrate=0.001, error=1950.303
>epoch=4, lrate=0.001, error=1948.927
>epoch=5, lrate=0.001, error=1948.788
>epoch=6, lrate=0.001, error=1948.779
>epoch=7, lrate=0.001, error=1948.780
>epoch=8, lrate=0.001, error=1948.781
>epoch=9, lrate=0.001, error=1948.782
>epoch=10, lrate=0.001, error=1948.782
>epoch=11, lrate=0.001, error=1948.782
>epoch=12, lrate=0.001, error=1948.782
>epoch=13, lrate=0.001, error=1948.782
>epoch=14, lrate=0.001, error=1948.782
>epoch=15, lrate=0.001, error=1948.782
>epoch=16, lrate=0.001, error=1948.782
>epoch=17, lrate=0.001, error=1948.782
>epoch=18, lrate=0.001, error=1948.782
>epoch=19, lrate=0.001, error=1948.782
>epoch=20, lrate=0.001, error=1948.782
>epoch=21, lrate=0.001, error=1948.782
>epoch=22, lrate=0.001, error=1948.782
>epoch=23, lrate=0.001, error=1948.782
>epoch=24, lrate=0.001, error=1948.782
>epoch=25, lrate=0.001, error=1948.

>epoch=12, lrate=0.001, error=1963.761
>epoch=13, lrate=0.001, error=1963.762
>epoch=14, lrate=0.001, error=1963.762
>epoch=15, lrate=0.001, error=1963.762
>epoch=16, lrate=0.001, error=1963.762
>epoch=17, lrate=0.001, error=1963.762
>epoch=18, lrate=0.001, error=1963.762
>epoch=19, lrate=0.001, error=1963.762
>epoch=20, lrate=0.001, error=1963.762
>epoch=21, lrate=0.001, error=1963.762
>epoch=22, lrate=0.001, error=1963.762
>epoch=23, lrate=0.001, error=1963.762
>epoch=24, lrate=0.001, error=1963.762
>epoch=25, lrate=0.001, error=1963.762
>epoch=26, lrate=0.001, error=1963.762
>epoch=27, lrate=0.001, error=1963.762
>epoch=28, lrate=0.001, error=1963.762
>epoch=29, lrate=0.001, error=1963.762
>epoch=30, lrate=0.001, error=1963.762
>epoch=31, lrate=0.001, error=1963.762
>epoch=32, lrate=0.001, error=1963.762
>epoch=33, lrate=0.001, error=1963.762
>epoch=34, lrate=0.001, error=1963.762
>epoch=35, lrate=0.001, error=1963.762
>epoch=36, lrate=0.001, error=1963.762
>epoch=37, lrate=0.001, e

>epoch=23, lrate=0.001, error=1902.009
>epoch=24, lrate=0.001, error=1902.009
>epoch=25, lrate=0.001, error=1902.009
>epoch=26, lrate=0.001, error=1902.009
>epoch=27, lrate=0.001, error=1902.009
>epoch=28, lrate=0.001, error=1902.009
>epoch=29, lrate=0.001, error=1902.009
>epoch=30, lrate=0.001, error=1902.009
>epoch=31, lrate=0.001, error=1902.009
>epoch=32, lrate=0.001, error=1902.009
>epoch=33, lrate=0.001, error=1902.009
>epoch=34, lrate=0.001, error=1902.009
>epoch=35, lrate=0.001, error=1902.009
>epoch=36, lrate=0.001, error=1902.009
>epoch=37, lrate=0.001, error=1902.009
>epoch=38, lrate=0.001, error=1902.009
>epoch=39, lrate=0.001, error=1902.009
>epoch=40, lrate=0.001, error=1902.009
>epoch=41, lrate=0.001, error=1902.009
>epoch=42, lrate=0.001, error=1902.009
>epoch=43, lrate=0.001, error=1902.009
>epoch=44, lrate=0.001, error=1902.009
>epoch=45, lrate=0.001, error=1902.009
>epoch=46, lrate=0.001, error=1902.009
>epoch=47, lrate=0.001, error=1902.009
>epoch=48, lrate=0.001, e

In [13]:
# evaluate algorithm with increased the number of hidden layer
n_folds = 5
l_rate = 0.001
n_epoch = 5
n_hidden = 20
scores = evaluate_algorithm(data, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/(len(scores))))

>epoch=0, lrate=0.001, error=5195.811
>epoch=1, lrate=0.001, error=5195.810
>epoch=2, lrate=0.001, error=5195.810
>epoch=3, lrate=0.001, error=5195.810
>epoch=4, lrate=0.001, error=5195.810
>epoch=0, lrate=0.001, error=5195.938
>epoch=1, lrate=0.001, error=5195.938
>epoch=2, lrate=0.001, error=5195.938
>epoch=3, lrate=0.001, error=5195.938
>epoch=4, lrate=0.001, error=5195.938
>epoch=0, lrate=0.001, error=5195.788
>epoch=1, lrate=0.001, error=5195.788
>epoch=2, lrate=0.001, error=5195.787
>epoch=3, lrate=0.001, error=5195.787
>epoch=4, lrate=0.001, error=5195.787
>epoch=0, lrate=0.001, error=5195.280
>epoch=1, lrate=0.001, error=5195.275
>epoch=2, lrate=0.001, error=5195.270
>epoch=3, lrate=0.001, error=5195.264
>epoch=4, lrate=0.001, error=5195.259
>epoch=0, lrate=0.001, error=5195.476
>epoch=1, lrate=0.001, error=5195.474
>epoch=2, lrate=0.001, error=5195.472
>epoch=3, lrate=0.001, error=5195.471
>epoch=4, lrate=0.001, error=5195.469
Scores: [22.863741339491916, 75.82755966127792, 75

I've tried increase learning rate, the number of hidde layer, epoch, and found that the best is:

n_folds = 5

l_rate = 0.001

n_epoch = 5

n_hidden = 5

with mean accuracy to be 75.396%